Aula 1 - Bibtex

In [1]:
import yaml

from python.helpers.pandas_helper import PandasHelper
from python.helpers.bibtex_helper import BibtexHelper
from yaml.loader import SafeLoader

with open('config.yaml') as f:
    config = yaml.load(f, Loader=SafeLoader)

bibtex_helper = BibtexHelper
pandas_helper = PandasHelper

df_acm = bibtex_helper.read_bibtex_files('dados_bibtex/ACM')
df_ieee = bibtex_helper.read_bibtex_files('dados_bibtex/IEEE')
df_sdc = bibtex_helper.read_bibtex_files('dados_bibtex/SDC')

df_acm = bibtex_helper.cleaner_columns(df=df_acm)
df_ieee = bibtex_helper.cleaner_columns(df=df_ieee)
df_sdc = bibtex_helper.cleaner_columns(df=df_sdc)

df_main = pandas_helper.concat_df(df_1=df_acm, df_2=df_ieee)
df_main = pandas_helper.concat_df(df_1=df_main, df_2=df_sdc)

df_main['title'] = df_main['title'].str.lower()

pandas_helper.save_df(df=df_main, file_type=config['file_saved_format_bibtex'], file_name_saved=config['name_file_saved_bibtex'])

Arquivos listado no Dataframe: ACM_34.bib
Arquivos listado no Dataframe: ACM_24.bib
Arquivos listado no Dataframe: ACM_13.bib
Arquivos listado no Dataframe: ACM_12.bib
Arquivos listado no Dataframe: ACM_26.bib
Arquivos listado no Dataframe: ACM_3.bib
Arquivos listado no Dataframe: ACM_5.bib
Arquivos listado no Dataframe: ACM_28.bib
Arquivos listado no Dataframe: ACM_18.bib
Arquivos listado no Dataframe: ACM_38.bib
Arquivos listado no Dataframe: ACM_36.bib
Arquivos listado no Dataframe: ACM_4.bib
Arquivos listado no Dataframe: ACM_31.bib
Arquivos listado no Dataframe: ACM_27.bib
Arquivos listado no Dataframe: ACM_29.bib
Arquivos listado no Dataframe: ACM_8.bib
Arquivos listado no Dataframe: ACM_25.bib
Arquivos listado no Dataframe: ACM_14.bib
Arquivos listado no Dataframe: ACM_7.bib
Arquivos listado no Dataframe: ACM_23.bib
Arquivos listado no Dataframe: ACM_1.bib
Arquivos listado no Dataframe: ACM_2.bib
Arquivos listado no Dataframe: ACM_30.bib
Arquivos listado no Dataframe: ACM_35.bib

Aula 2 - CSV (JCR e SCIMAGOJR)

In [2]:

import yaml

from python.helpers.pandas_helper import PandasHelper
from python.helpers.bibtex_helper import BibtexHelper
from yaml.loader import SafeLoader

with open('config.yaml') as f:
    config = yaml.load(f, Loader=SafeLoader)

bibtex_helper = BibtexHelper
pandas_helper = PandasHelper()

from python.helpers.csv_helper import CSVHelper


csv_helper = CSVHelper

df_scimagojr = csv_helper.read_csv_to_pandas('dados_rank/scimagojr_2020.csv')
df_jcs = csv_helper.read_csv_to_pandas('dados_rank/jcs_2020.csv')

# Dropa colunas que só possuem NaN
df_scimagojr.dropna(how='all', axis=1, inplace=True)
df_jcs.dropna(how='all', axis=1, inplace=True)

# Passa o nome das colunas pra letra minuscula e checa o nome e altera
csv_helper.cleaner_columns_csv(df=df_scimagojr, column_check='title', column_new_name='title')
csv_helper.cleaner_columns_csv(df=df_jcs, column_check='title', column_new_name='title')

In [3]:
# Passa todos os valores do campo "title" para letra minuscula
df_jcs['title'] = df_jcs['title'].str.lower()
df_scimagojr['title'] = df_scimagojr['title'].str.lower()

In [4]:
df_merged_csv = pandas_helper.merge_dataframes(df_1=df_scimagojr, df_2=df_jcs, column_to_merge='title')

In [5]:
df_merged_csv.head(5)

,rank_x,sourceid,title,type,issn,sjr,sjr_best_quartile,h_index,total_docs__2020,total_docs__3years,...,ref___doc_,country,region,publisher,coverage,categories,rank_y,total_cites,journal_impact_factor,eigenfactor_score
0,1,28773,ca-a cancer journal for clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,...,"73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),1,"55,868",508.702,0.10514
1,2,19434,mmwr recommendations and reports,journal,"10575987, 15458601","40,949",Q1,143,10,9,...,"129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...,14,"3,288",55.857,0.01123
2,3,20315,nature reviews molecular cell biology,journal,"14710072, 14710080","37,461",Q1,431,115,338,...,"73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1),2,"58,477",94.444,0.07548
3,4,29431,quarterly journal of economics,journal,"00335533, 15314650","34,573",Q1,259,40,110,...,"68,33",United Kingdom,Western Europe,Oxford University Press,1886-2020,Economics and Econometrics (Q1),215,"38,427",15.563,0.06169
4,5,21100812243,nature reviews materials,journal,20588437,"32,011",Q1,108,92,264,...,"115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag...",7,"19,887",66.308,0.05677


In [6]:
# Muda os nomes de algumas colunas
df_merged_csv.rename(columns={'rank_x': 'scimago_value'}, inplace=True)
df_merged_csv.rename(columns={'rank_y': 'jcr_value'}, inplace=True)

In [7]:
df_merged_bibtex_csv = pandas_helper.merge_dataframes(df_1=df_main, df_2=df_merged_csv, column_to_merge='title')

In [8]:
df_merged_bibtex_csv = pandas_helper.remove_duplicate(df=df_merged_bibtex_csv)

In [9]:
df_filter = pandas_helper.filter_column(df=df_merged_csv, column_filter=config['columns_filters'])

In [10]:
df_filter.head(10)

,scimago_value,sourceid,title,type,issn,sjr,sjr_best_quartile,h_index,total_docs__2020,total_docs__3years,...,ref___doc_,country,region,publisher,coverage,categories,jcr_value,total_cites,journal_impact_factor,eigenfactor_score
0,1,28773,ca-a cancer journal for clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,...,"73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),1,"55,868",508.702,0.10514
2,3,20315,nature reviews molecular cell biology,journal,"14710072, 14710080","37,461",Q1,431,115,338,...,"73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1),2,"58,477",94.444,0.07548
73,78,20425,nature reviews drug discovery,journal,"14741784, 14741776","9,921",Q1,328,247,676,...,"32,17",United Kingdom,Western Europe,Nature Publishing Group,2002-2020,Drug Discovery (Q1); Medicine (miscellaneous) ...,4,"41,989",84.694,0.04822
4,5,21100812243,nature reviews materials,journal,20588437,"32,011",Q1,108,92,264,...,"115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag...",7,"19,887",66.308,0.05677
5,7,18434,cell,journal,"00928674, 10974172","26,304",Q1,776,572,1690,...,"61,79",United States,Northern America,Cell Press,1974-2020,"Biochemistry, Genetics and Molecular Biology (...",33,"320,407",41.582,0.52696


In [11]:
pandas_helper.save_df(df=df_filter, file_type=config['file_saved_format_rank'], file_name_saved=config['name_file_saved_rank'])

Arquivo salvo: resultados/results_rank.yaml
